# Libraries

In [2]:
import pandas as pd
import numpy as np
import math
import pickle

from scipy import stats
import scipy.io
from scipy.spatial.distance import pdist
from scipy.linalg import cholesky
from scipy.io import loadmat

import matlab.engine as engi
import matlab as mat

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,roc_auc_score,recall_score,precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from src import SMOTE
from src import CFS
from src import metrices

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path

import matplotlib.pyplot as plt

# Start matlab service

In [3]:
eng = engi.start_matlab()
eng.addpath(r'src/matlab_CTKCCA/',nargout=0)
eng.addpath(r'src/matlab_KS/',nargout=0)

# variables

In [4]:
result_path = 'result/result.csv'
repeats = 20
ratio = 0.1
lrank = 70
reg = 1E-5

# Data loading and Normalizing Data

In [5]:
def load_data(project):
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    commit_guru_path = 'data/commit_guru/' + project + '.csv'
    understand_df = pd.read_csv(understand_path)
    understand_df = understand_df.dropna(axis = 1,how='all')
    cols_list = understand_df.columns.values.tolist()
    for item in ['Kind', 'Name','commit_hash', 'Bugs']:
        if item in cols_list:
            cols_list.remove(item)
            cols_list.insert(0,item)
    understand_df = understand_df[cols_list]
    commit_guru_df = pd.read_csv(commit_guru_path)
    cols = understand_df.columns.tolist()
    
    commit_guru_df = commit_guru_df.drop(labels = ['parent_hashes','author_name','author_name',
                                                   'author_email','fileschanged','author_date',
                                                   'author_date_unix_timestamp', 'commit_message',
                                                  'classification', 'fix', 'contains_bug','fixes',],axis=1)

#     print(commit_guru_df.columns)
    understand_df = understand_df.drop_duplicates(cols[4:len(cols)])
    df = understand_df.merge(commit_guru_df,on='commit_hash')
    cols = df.columns.tolist()
    cols = cols[1:] + [cols[0]]
    df = df[cols]
    for item in ['Kind', 'Name','commit_hash']:
        if item in cols:
            df = df.drop(labels = [item],axis=1)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
#     s_df,s_cols = apply_cfs(df)
    y = df.Bugs
    X = df.drop('Bugs',axis = 1)
    cols = X.columns
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    df = pd.concat([X,y],axis = 1)
    return df

def apply_smote(df):
    cols = df.columns
    smt = SMOTE.smote(df)
    df = smt.run()
    df.columns = cols
    return df

def apply_cfs(df):
        y = df.Bugs.values
        X = df.drop(labels = ['Bugs'],axis = 1)
        X = X.values
        selected_cols = CFS.cfs(X,y)
        cols = df.columns[[selected_cols]].tolist()
        cols.append('Bugs')
        return df[cols],cols

# Matlab integration

# Teting using original Data

## get train test data

In [10]:
precision_list = {}
recall_list = {}
pf_list = {}
f1_list = {}
g_list = {}
auc_list = {}
proj_df = pd.read_csv('projects.csv')
projects = proj_df.repo_name.tolist()
for s_project in projects:
    try:
        if s_project not in precision_list.keys():
            precision_list[s_project] = {}
            recall_list[s_project] = {}
            pf_list[s_project] = {}
            f1_list[s_project] = {}
            g_list[s_project] = {}
            auc_list[s_project] = {} 
        source_df = load_data(s_project)
        source_df = apply_smote(source_df)
#         print(source_df)
        train_y = source_df.Bugs
        train_X = source_df.drop('Bugs',axis = 1)
        for d_project in projects:
            try:
                target_df = load_data(d_project)
                test_y = target_df.Bugs
                test_X = target_df.drop('Bugs',axis = 1)
                #Training Model & Predicting
                clf = LogisticRegression()
                clf.fit(train_X,train_y)
                predicted = clf.predict(test_X)
                # Calculating metrics
                abcd = metrices.measures(test_y,predicted)
                pf = abcd.get_pf()
                recall = abcd.calculate_recall()
                precision = abcd.calculate_precision()
                f1 = abcd.calculate_f1_score()
                g_score = abcd.get_g_score()
                auc = roc_auc_score(test_y,predicted)
                # Storing Performance scores
                precision_list[s_project][d_project] = precision
                recall_list[s_project][d_project] = recall
                pf_list[s_project][d_project] = pf
                f1_list[s_project][d_project] = f1
                g_list[s_project][d_project] = g_score
                auc_list[s_project][d_project] = auc
                print(classification_report(test_y,predicted))
            except Exception as e:
                print(e)
                break
    except Exception as e:
        print(e)
        break
final_result = {}
final_result['precision'] = precision_list
final_result['recall'] = recall_list
final_result['pf'] = pf_list
final_result['f1'] = f1_list
final_result['g'] = g_list
final_result['auc'] = auc_list
with open('results/Performance/Bellwether_100.pkl', 'wb') as handle:
    pickle.dump(final_result, handle, protocol=pickle.HIGHEST_PROTOCOL)
bell_performance = {}
for metric in final_result.keys():
    if metric not in bell_performance.keys():
        bell_performance[metric] = {}
    for project in final_result[metric].keys():
        bell_performance[metric][project] = np.median(list(final_result[metric][project].values()))
bell_performance_df = pd.DataFrame.from_dict(bell_performance)

              precision    recall  f1-score   support

           0       0.85      0.80      0.82       462
           1       0.57      0.65      0.61       189

    accuracy                           0.76       651
   macro avg       0.71      0.72      0.72       651
weighted avg       0.77      0.76      0.76       651

              precision    recall  f1-score   support

           0       0.59      0.73      0.65       838
           1       0.63      0.48      0.55       826

    accuracy                           0.60      1664
   macro avg       0.61      0.60      0.60      1664
weighted avg       0.61      0.60      0.60      1664

              precision    recall  f1-score   support

           0       0.77      0.72      0.74     12345
           1       0.43      0.50      0.46      5330

    accuracy                           0.65     17675
   macro avg       0.60      0.61      0.60     17675
weighted avg       0.67      0.65      0.66     17675

              preci

              precision    recall  f1-score   support

           0       0.82      0.67      0.74      2142
           1       0.56      0.74      0.64      1209

    accuracy                           0.70      3351
   macro avg       0.69      0.71      0.69      3351
weighted avg       0.73      0.70      0.70      3351

              precision    recall  f1-score   support

           0       0.66      0.83      0.74      1553
           1       0.58      0.34      0.43      1032

    accuracy                           0.64      2585
   macro avg       0.62      0.59      0.58      2585
weighted avg       0.63      0.64      0.61      2585

              precision    recall  f1-score   support

           0       0.63      0.88      0.74      3161
           1       0.76      0.41      0.54      2788

    accuracy                           0.66      5949
   macro avg       0.69      0.65      0.64      5949
weighted avg       0.69      0.66      0.64      5949

              preci

              precision    recall  f1-score   support

           0       0.83      0.76      0.79      2471
           1       0.32      0.44      0.37       655

    accuracy                           0.69      3126
   macro avg       0.58      0.60      0.58      3126
weighted avg       0.73      0.69      0.70      3126

              precision    recall  f1-score   support

           0       0.84      0.94      0.89      2697
           1       0.52      0.26      0.35       653

    accuracy                           0.81      3350
   macro avg       0.68      0.60      0.62      3350
weighted avg       0.78      0.81      0.78      3350

              precision    recall  f1-score   support

           0       0.92      0.86      0.89      6152
           1       0.42      0.60      0.49      1081

    accuracy                           0.82      7233
   macro avg       0.67      0.73      0.69      7233
weighted avg       0.85      0.82      0.83      7233

              preci

              precision    recall  f1-score   support

           0       0.81      0.51      0.62       462
           1       0.37      0.70      0.48       189

    accuracy                           0.56       651
   macro avg       0.59      0.61      0.55       651
weighted avg       0.68      0.56      0.58       651

              precision    recall  f1-score   support

           0       0.56      0.64      0.60       838
           1       0.57      0.48      0.52       826

    accuracy                           0.56      1664
   macro avg       0.56      0.56      0.56      1664
weighted avg       0.56      0.56      0.56      1664

              precision    recall  f1-score   support

           0       0.75      0.57      0.65     12345
           1       0.36      0.57      0.44      5330

    accuracy                           0.57     17675
   macro avg       0.56      0.57      0.54     17675
weighted avg       0.63      0.57      0.58     17675

              preci

              precision    recall  f1-score   support

           0       0.72      0.74      0.73      2142
           1       0.52      0.49      0.50      1209

    accuracy                           0.65      3351
   macro avg       0.62      0.62      0.62      3351
weighted avg       0.65      0.65      0.65      3351

              precision    recall  f1-score   support

           0       0.66      0.82      0.73      1553
           1       0.57      0.36      0.44      1032

    accuracy                           0.64      2585
   macro avg       0.62      0.59      0.59      2585
weighted avg       0.63      0.64      0.62      2585

              precision    recall  f1-score   support

           0       0.64      0.61      0.62      3161
           1       0.58      0.60      0.59      2788

    accuracy                           0.61      5949
   macro avg       0.61      0.61      0.61      5949
weighted avg       0.61      0.61      0.61      5949

              preci

...

Only one class present in y_true. ROC AUC score is not defined in that case.
              precision    recall  f1-score   support

           0       0.75      0.83      0.79       462
           1       0.44      0.32      0.37       189

    accuracy                           0.68       651
   macro avg       0.59      0.58      0.58       651
weighted avg       0.66      0.68      0.67       651

              precision    recall  f1-score   support

           0       0.68      0.57      0.62       838
           1       0.63      0.72      0.67       826

    accuracy                           0.65      1664
   macro avg       0.65      0.65      0.65      1664
weighted avg       0.65      0.65      0.65      1664

              precision    recall  f1-score   support

           0       0.80      0.46      0.58     12345
           1       0.37      0.74      0.49      5330

    accuracy                           0.54     17675
   macro avg       0.59      0.60      0.54     176

              precision    recall  f1-score   support

           0       0.73      0.77      0.75      2142
           1       0.55      0.50      0.52      1209

    accuracy                           0.67      3351
   macro avg       0.64      0.63      0.63      3351
weighted avg       0.66      0.67      0.67      3351

              precision    recall  f1-score   support

           0       0.68      0.80      0.73      1553
           1       0.59      0.43      0.49      1032

    accuracy                           0.65      2585
   macro avg       0.63      0.61      0.61      2585
weighted avg       0.64      0.65      0.64      2585

              precision    recall  f1-score   support

           0       0.60      0.96      0.74      3161
           1       0.86      0.27      0.41      2788

    accuracy                           0.64      5949
   macro avg       0.73      0.62      0.58      5949
weighted avg       0.72      0.64      0.59      5949

              preci

              precision    recall  f1-score   support

           0       0.88      0.80      0.84      2471
           1       0.43      0.58      0.49       655

    accuracy                           0.75      3126
   macro avg       0.65      0.69      0.66      3126
weighted avg       0.78      0.75      0.76      3126

              precision    recall  f1-score   support

           0       0.81      0.94      0.87      2697
           1       0.20      0.06      0.10       653

    accuracy                           0.77      3350
   macro avg       0.50      0.50      0.48      3350
weighted avg       0.69      0.77      0.72      3350

              precision    recall  f1-score   support

           0       0.92      0.92      0.92      6152
           1       0.53      0.52      0.52      1081

    accuracy                           0.86      7233
   macro avg       0.72      0.72      0.72      7233
weighted avg       0.86      0.86      0.86      7233

              preci

Only one class present in y_true. ROC AUC score is not defined in that case.
              precision    recall  f1-score   support

           0       0.73      0.87      0.79       462
           1       0.38      0.20      0.26       189

    accuracy                           0.67       651
   macro avg       0.55      0.53      0.53       651
weighted avg       0.63      0.67      0.64       651

              precision    recall  f1-score   support

           0       0.54      0.92      0.68       838
           1       0.72      0.21      0.33       826

    accuracy                           0.57      1664
   macro avg       0.63      0.57      0.50      1664
weighted avg       0.63      0.57      0.51      1664

              precision    recall  f1-score   support

           0       0.90      0.80      0.85     12345
           1       0.64      0.80      0.71      5330

    accuracy                           0.80     17675
   macro avg       0.77      0.80      0.78     176

              precision    recall  f1-score   support

           0       0.76      0.78      0.77      2142
           1       0.59      0.57      0.58      1209

    accuracy                           0.70      3351
   macro avg       0.68      0.67      0.67      3351
weighted avg       0.70      0.70      0.70      3351

              precision    recall  f1-score   support

           0       0.67      0.75      0.71      1553
           1       0.54      0.43      0.48      1032

    accuracy                           0.63      2585
   macro avg       0.60      0.59      0.59      2585
weighted avg       0.62      0.63      0.62      2585

              precision    recall  f1-score   support

           0       0.58      0.94      0.72      3161
           1       0.77      0.23      0.35      2788

    accuracy                           0.61      5949
   macro avg       0.68      0.58      0.53      5949
weighted avg       0.67      0.61      0.54      5949

              preci